### Installation of libraries

In [ ]:
!pip install -U autogen-agentchat autogen-ext[openai]
!pip install torch transformers sentencepiece

In [ ]:
!pip install autogen

### Initial setup

API key setup

In [19]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/WebUI-7k

### Multi Agent Setup

#### Fine tuned Agents

Wrapper agent for autogen

Semantic, Contrast Agents

In [ ]:
from agents.semantic_agent.agent import SemanticAgent
from agents.contrast_agent.agent import ContrastAgent
from agents.image_captioning_agent.agent import ImageCaptioningAgent

semantic_model = SemanticAgent(model_dir="trusha88/t5-semantic-agent")
contrast_model = ContrastAgent(model_dir="virajns2/contrast-violation-t5")
image_caption_model = ImageCaptioningAgent()

In [5]:
from agents.axe_violations_agent.agent import AxeViolationsAgent

axe_agent = AxeViolationsAgent()

#### GPT based Agents

In [9]:
from autogen import AssistantAgent

In [10]:
# Visually Impaired Agent
visually_impaired_agent = AssistantAgent(
    name="VisuallyImpairedAgent",
    system_message="You are a screen‑reader user. Given the following combined accessibility summary from the SemanticAgent and ContrastAgent, analyze it and respond with any additional issues or validations as you navigate the page. Include explicit references to each semantic and contrast finding.",
    llm_config={"model": "gpt-4", "temperature": 0}
)

In [11]:
# Motor-Impaired Agent
motor_impaired_agent = AssistantAgent(
    name="MotorImpairedAgent",
    system_message="You are a keyboard‑only user. Given the combined accessibility summary above, walk through the page structure and identify keyboard navigation barriers (e.g., tabindex issues, missing focus styles). Refer back to each semantic/contrast point in your response.",
    llm_config={"model": "gpt-4", "temperature": 0}
)


In [12]:
# Color-Blind Agent
color_blind_agent = AssistantAgent(
    name="ColorBlindAgent",
    system_message="You are a color‑blind user. Using the combined summary, assess whether the listed contrast ratios and semantic issues affect your ability to distinguish page elements. Call out any color‑related problems or confirm that the reported contrast ratio is sufficient.",
    llm_config={"model": "gpt-4", "temperature": 0}
)

In [13]:
fixing_agent = AssistantAgent(
    name="FixingAgent",
    system_message="You are the final‑stage accessibility engineer. Given the full conversation history—including the semantic and contrast summaries and each simulation agent’s findings—produce a consolidated list of code‑level fixes. For each issue, reference which agent(s) raised it, then provide the minimal HTML/CSS/ARIA snippet needed to resolve it.",
    llm_config={"model": "gpt-4", "temperature": 0}
)

### Configure Agent Interactions

In [ ]:
from autogen.agentchat import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager
import json

# A silent proxy for driving “human” turns
user_proxy = UserProxyAgent(
    name="UserProxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=1,
    code_execution_config={"use_docker": False}
)

# Load your UI JSON once
with open("test_data/test_file.json", "r", encoding="utf-8") as f:
    ui_json = json.load(f)

# Make sure ui_str is a JSON string:
if isinstance(ui_json, dict):
    ui_str = json.dumps(ui_json)
else:
    ui_str = ui_json

print(json.dumps(ui_json, indent=4))

In [ ]:
import json

# 1) Make sure your JSON is a string
if isinstance(ui_json, dict):
    ui_str = json.dumps(ui_json)
else:
    ui_str = ui_json

# 2) Wrap your T5 agents exactly as before
class ChatWrapperAgent(AssistantAgent):
    def __init__(self, name: str, t5_agent):
        super().__init__(name=name)
        self.t5_agent = t5_agent

    def generate_reply(self, messages, sender=None, **kwargs):
        # take the content of the *first* message as raw JSON
        raw_json = messages[0]["content"]
        return self.t5_agent.handle(raw_json)

semantic_wrapper = ChatWrapperAgent("semantic-agent", semantic_model)
contrast_wrapper = ChatWrapperAgent("contrast-agent", contrast_model)
axe_wrapper = ChatWrapperAgent("axe-violations-agent", axe_agent)
image_captioning_wrapper = ChatWrapperAgent("image-captioning-agent", image_caption_model)

# 3) Manually invoke them on the same single‐element history:
history = [{"role": "user", "content": ui_str}]

semantic_summary  = semantic_wrapper.generate_reply(history)
contrast_summary = contrast_wrapper.generate_reply(history)
axe_violations_summary = axe_wrapper.generate_reply(history)
image_captioning_summary = image_captioning_wrapper.generate_reply(history)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:679: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


In [ ]:
combined_summary = (
    f"SemanticAgent: {semantic_summary}\n\n"
    f"ContrastAgent: {contrast_summary}\n\n"
    f"AxeViolationsAgent: {axe_violations_summary}\n\n"
    f"ImageCaptioningAgent: {image_captioning_summary}\n\n"
)

print("\n=== Combined Summary ===")
print(combined_summary)


=== Combined Summary ===
SemanticAgent: On page 1656130839224 (viewport 1920-1080) we detected the following semantic violations: Violation 1 — **landmark-one-main** (impact: moderate) • Description: Ensures the document has a main landmark. • Recommendation: Document must have one main landmark. See https://dequeuniversity.com/rules/axe/3.5/landmark-one-main?application=axe-puppeteer • Detail: Element "h4>" at html failed: Document does not have a main landmark. Violation 2 — **page-has-heading-one** (impact: moderate) • Description: Ensure that the page, or at least one of its frames contains a level-one heading. • Recommendation: Page must contain a level-one heading. See https://dequeuniversity.com/rules/axe/3.5/page-has-heading-one?application=axe-puppeteer • Detail: Element "html>" at html failed: Page must have a level

ContrastAgent: The link element has a foreground color of RGB(255,255,255) and background color of RGB(52,58,64) resulting in a contrast ratio of 11.51.

AxeVio

In [ ]:
from autogen.agentchat import UserProxyAgent, GroupChat, GroupChatManager

# 1) Rebuild the echo agents around your summaries
class EchoAgent(AssistantAgent):
    def __init__(self, name: str, summary: str):
        super().__init__(name=name)
        self.summary = summary

    def generate_reply(self, messages=None, sender=None, **kwargs):
        return self.summary

echo_semantic = EchoAgent("semantic-agent", semantic_summary)
echo_contrast = EchoAgent("contrast-agent", contrast_summary)
echo_axe_violations = EchoAgent("axe-violations-agent", axe_violations_summary)
echo_image_captioning = EchoAgent("image-captioning-agent", image_captioning_summary)

# 2) Configure your proxy
user_proxy = UserProxyAgent(
    name="UserProxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=1,
    code_execution_config={"use_docker": False}
)

# 3) Build the GroupChat with explicit round‑robin ordering
group_chat = GroupChat(
    agents=[
        user_proxy,
        echo_semantic,
        echo_contrast,
        echo_axe_violations,
        echo_image_captioning,
        visually_impaired_agent,
        motor_impaired_agent,
        color_blind_agent,
        fixing_agent
    ],
    messages=[],  # no pre‑seed
    max_round=8,
    speaker_selection_method="round_robin",
    allow_repeat_speaker=False
)

# 4) Bind a new manager
manager = GroupChatManager(
    groupchat=group_chat,
    llm_config={"model": "gpt-4", "temperature": 0}
)

# 5) Kick off with your combined summary; max_turns = 6 agents after the proxy
chat_result = user_proxy.initiate_chat(
    manager,
    message=combined_summary,
    clear_history=True,
    max_turns=1
)
